# Net Primary Production (NPP) - estimation of annual anomalies and trends over the last 20 years using R software: a case study in Estonia
tutors: Bruno Montibeller and Iuliia Burdun <br>
contacts bruno.montibeller@ut.ee


## Introduction
**The goal of this workshop is to teach the participants how to work with spatial data in R programming software by applying the Mann-Kendall (MK) trend test and calculate Theil–Sen values on a gridded net primary production data.**

Net primary production (NPP) can be defined as the quantity of carbon assimilated per unit of area over a given period of time. It is calculated through the subtraction of the plant respiration rates from the gross primary production rates. The NPP data (named __[MOD17A3HGF](https://lpdaac.usgs.gov/products/mod17a3hgfv006/#tools)__) that we will be using is based on the data acquired by the Moderate Resolution Imaging Spectroradiometer (MODIS) remote sensor onboard of the Terra satellite. The data is calculated on annual basis using the gross primary production product of the same sensor, annual rates of growth respiration and annual maintenance respiration of live cells in woody. Additional info can be found on the __[user guide](https://lpdaac.usgs.gov/documents/495/MOD17_User_Guide_V6.pdf)__. 

MK trend test has been widely applied to infer about the temporal trends in time series of environmental variables. MK is a nonparametric test, therefore, it does not require normal distribution of the data, but it does assume no autocorrelation in the time series. The MK test has as null hypothesis that the time series has no trend, and the alternative hypothesis is that it does have a trend (upward or downward). 

The MK trend test first calculates the differences between all pairs of earlier and later data points in the time series. After that, it calculates the variance of these differences, which is posteriorly used to calculate the Z test value. Finally, the statistical significance of the trend is quantified by calculating the probability associated with the normalized Z test. If Z > 0, it indicates an increasing trend and if Z < 0 it indicates a decreasing trend. Usually, the trends are defined as significant using 95% confidence level. In addition to the trend calculation, it is also possible to quantify the magnitude of the trends. The magnitudes can be estimated by using the nonparametric Sen statistic, more specifically, the Theil–Sen estimator which is given by the median of the slopes of each pair of points. To calculate the Theil–Sen, the times series data is ordered accordingly to the time (as function of time) and a confidence interval is provided for each slope value (same as in the MK test).

## Data
* __[Estonian boundaries](https://gadm.org/data.html)__
* __[Estonian forest cover](https://doi.org/10.1038/s41893-020-00609-y)__
* __[MODIS NPP](https://lpdaacsvc.cr.usgs.gov/appeears/)__



## R libraries 
* __[terra](https://cran.r-project.org/web/packages/terra/terra.pdf)__ - methods for spatial data analysis with raster and vector data
* __[raster](https://cran.r-project.org/web/packages/raster/raster.pdf)__ - reading, writing, manipulating, analyzing and modeling of spatial data.
* __[tmap](https://cran.r-project.org/web/packages/tmap/tmap.pdf)__ - create thematic maps
* __[sf](https://cran.r-project.org/web/packages/sf/sf.pdf)__ - support for simple features, a standardized way to encode spatial vector data
* __[viridis](https://cran.r-project.org/web/packages/viridis/viridis.pdf)__ - color maps designed to improve graph readability for readers with common forms of color blindness and/or color vision deficiency
* __[rgdal](https://cran.r-project.org/web/packages/rgdal/rgdal.pdf)__ - provides access to projection/transformation operations
* __[tidyverse](https://www.tidyverse.org/)__ - set of packages designed for data science 
* __[tidyr](https://cran.r-project.org/web/packages/tidyr/tidyr.pdf)__  - tools for changing the shapeand hierarchy of a dataset
* __[wql](https://cran.r-project.org/web/packages/wql/wql.pdf)__ - functions useful for analysis of similar-frequency time series data
* __[plyr](https://cran.r-project.org/web/packages/plyr/plyr.pdf)__ - helps summarize data quickly
* __[maditr](https://cran.r-project.org/web/packages/maditr/maditr.pdf)__  - provides pipe-style interface for data.table package
* __[gifski](https://cran.r-project.org/web/packages/gifski/gifski.pdf)__  - converts images to GIF animations 
* __[curl](https://cran.r-project.org/web/packages/curl/curl.pdf)__ - download data from web

## Workflow


In [ ]:
#Install libraries
install.packages(c("terra","raster","tmap","sf","viridis", "rgdal", "tidyverse", "tidyr", 
                       "wql", "plyr", "maditr", "gifski", "curl"), quiet = TRUE) 

In [ ]:
# Load libraries
require(terra)
require(raster)
require(tmap)
require(sf)
require(viridis)
require(rgdal)
require(tidyverse)
require(tidyr)
require(wql)
require(plyr)
require(maditr)
require(gifski)
require(curl) 

In [3]:
.libPaths("C:/Users/burdu/OneDrive - Tartu Ülikool/Iuliia Burdun Tartu/ESP workshop/jupyter_R")
.libPaths()

[1] "C:/Users/burdu/OneDrive - Tartu Ülikool/Iuliia Burdun Tartu/ESP workshop/jupyter_R"
[2] "C:/Users/burdu/miniconda3/envs/espworkshop/Lib/R/library"

In [4]:
install.packages("terra", quiet = TRUE) 

also installing the dependencies 'lattice', 'sp', 'raster'




  There is a binary version available but the source version is later:
      binary source needs_compilation
terra  1.2-5 1.2-10              TRUE

  Binaries will be installed
package 'lattice' successfully unpacked and MD5 sums checked
package 'sp' successfully unpacked and MD5 sums checked
package 'raster' successfully unpacked and MD5 sums checked
package 'terra' successfully unpacked and MD5 sums checked


In [ ]:
require(terra)

Loading required package: terra
Warning message:
"package 'terra' was built under R version 3.6.3"

In [1]:
R.version

               _                           
platform       x86_64-w64-mingw32          
arch           x86_64                      
os             mingw32                     
system         x86_64, mingw32             
status                                     
major          3                           
minor          6.1                         
year           2019                        
month          07                          
day            05                          
svn rev        76782                       
language       R                           
version.string R version 3.6.1 (2019-07-05)
nickname       Action of the Toes          